### Import statements

In [ ]:
%run ../functions/startup.py
random.seed(4)

from _funct_annotat import Labels_proc_NEST
lab_p = Labels_proc_NEST()

In [ ]:
def get_parent_dir(directory):
    import os
    return os.path.dirname(directory)

current_dirs_parent = get_parent_dir(os.getcwd())

In [ ]:
path_base = str(current_dirs_parent)
folder = '/data_examples/NEST_data/'  # Select data folder
labels = 'labels/'

path_all = path_base + folder
save_path = path_all     # Destination folder to for labelled data

# Merge single anomalies to collective anomalies for 2 sensors

In [ ]:
# Select Case
Case = 1      # case 1: GAK (p3,p4),       case 2: Pressure T1 T2 (p1,p2)

if Case == 1:
    lab1 = '_labels_timep3'
    lab2 = '_labels_timep4'
    lab3 = '_labels_timep3p4'
if Case == 2:
    lab1 = '_labels_timep1'
    lab2 = '_labels_timep2'
    lab3 = '_labels_timep1p2'

In [ ]:
### Load multiple sensors anomalies

data_all_labels = lab_p.data_labels_load(path_base,folder, labels, lab3)
data_all_labels = lab_p.process_data(data_all_labels)


In [ ]:
### Load single sensors anomalies

data_1 = lab_p.data_labels_load(path_base,folder, labels, lab1)
data_1 = lab_p.process_data(data_1)

data_2 = lab_p.data_labels_load(path_base,folder, labels, lab2)
data_2 = lab_p.process_data(data_2)

In [ ]:
### check if dates coincide and how many anomalies are different

print(data_1.shape[0] == (np.sum(data_1.datetime == data_2.datetime)))
print('Missing anomalies', data_all_labels.shape[0] - np.sum(data_1.Anomaly == data_all_labels.Anomaly))
print('Missing anomalies', data_all_labels.shape[0] - np.sum(data_2.Anomaly == data_all_labels.Anomaly))

### merge anomalies

In [ ]:
Anom_all = data_all_labels.Anomaly.copy()
count = 0
for i in range (data_1.shape[0]):
    if ((data_1.datetime[i] == data_all_labels.datetime[i]) & (data_1.Anomaly[i] == 1)):
        count = count +1
        Anom_all[i] = 1
print('Total added anomalies:', count)

# we do not initialize again Amoal_all
count = 0
for i in range (data_2.shape[0]):
    if ((data_2.datetime[i] == data_all_labels.datetime[i]) & (data_2.Anomaly[i] == 1)):
        count = count +1
        Anom_all[i] = 1
print('Total added anomalies:', count)

print('Fraction of combined anomalies: ', np.sum(Anom_all)/data_all_labels.shape[0])

In [ ]:
data_all_labels['Tot_anomalies'] = Anom_all
data_all_labels.drop(columns = ['index', 'datetime'], inplace=True)

In [ ]:
# # Save data as pickle
if Case == 1:
    name = 'GAK_Anomaly_collective' 
else:
    name = 'T1_T2_Anomaly_collective' 

file = open(path_all + labels + name, 'wb')   # open a file, where you want to store the data
pickle.dump(data_all_labels, file, protocol=4)   # dump information to that file
file.close()   # close the file

----------

# OPTIONAL: merge the 2 collective cases

## Load data

In [ ]:
name1 = 'GAK_Anomaly_collective'
name2 = 'T1_T2_Anomaly_collective'


# Load data file  - Working with collective anomalies
    
file = open(path_all + labels + name1, 'rb')  # open a file, where you stored the pickled data
df1 = pickle.load(file)   # dump information to that file
file.close()   # close the file


file = open(path_all + labels + name2, 'rb')  # open a file, where you stored the pickled data
df2 = pickle.load(file)   # dump information to that file
file.close()   # close the file


In [ ]:
df1 = lab_p.process_data_coll(df1)
df2 = lab_p.process_data_coll(df2)

In [ ]:
print('Missing anomalies', df1.Tot_anomalies.shape[0] - np.sum(df1.Tot_anomalies == df2.Tot_anomalies))

#### Merge anomalies

In [ ]:
TT = df1.Tot_anomalies + df2.Tot_anomalies

for i, x in enumerate(TT):
    if x >= 1: TT[i] = 1
        
list_df = list(zip(df1.day, df1.time, df2.p1, df2.p2, df1.p3, df1.p4, TT))

merg_data_df = pd.DataFrame(list_df, columns=['day', 'time', 'p1', 'p2', 'p3', 'p4', 'Tot_anomalies'])

In [ ]:
print('Fraction of anomalies: ', np.sum(merg_data_df['Tot_anomalies'] ==1)/merg_data_df.shape[0])

In [ ]:
# # Save data as pickle

name = 'NEST_Anomaly_collective' 

file = open(path_all + labels + name, 'wb')   # open a file, where you want to store the data
pickle.dump(merg_data_df, file, protocol=4)   # dump information to that file
file.close()   # close the file

# Data manipulation - SUBSAMPLING


In [ ]:
# Load data file  - Working with collective anomalies

name = 'NEST_Anomaly_collective' 
    
file = open(path_all + labels + name, 'rb')  # open a file, where you stored the pickled data
df1 = pickle.load(file)   # dump information to that file
file.close()   # close the file

df1.fillna(0, inplace=True)
print ('Data shape', df1.shape)
print (df1.keys())

print('Fraction of anomalies:',np.sum(df1.Tot_anomalies)/df1.shape[0])

In [ ]:
step = 6

df_subs = df1.copy(deep=True)
df_subs = df_subs.loc[::step]    # start:stop:step   , subsample every minute

print('Fraction of anomalies:', np.sum(df_subs.Tot_anomalies)/df_subs.shape[0])

In [ ]:
# # Save data as pickle

name = name + '_sub_' + str(step)

file = open(path_all + labels + name, 'wb')   # open a file, where you want to store the data
pickle.dump(df_subs, file, protocol=4)   # dump information to that file
file.close()   # close the file